#01. 트렌스포머 설치 및 필요 패키지 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#02. 미리 생성해 둔 학습 데이터 불러오기  
https://zetawiki.com/wiki/한국어_말뭉치_모음  
 - 챗봇 트레이닝용 문답 페어 11,876개  
 - 일상다반서 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링

In [ ]:
!wget -c https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv
#최초 데이터 수집 시에만 실행

--2022-10-30 05:15:05--  https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv [following]
--2022-10-30 05:15:06--  https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
import pandas as pd
df = pd.read_csv('ChatbotData.csv')
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [ ]:
df.tail()

,Q,A,label
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2
11822,힘들어서 결혼할까봐,도피성 결혼은 하지 않길 바라요.,2


In [ ]:
df['label'].value_counts()

0    5290
1    3570
2    2963
Name: label, dtype: int64

#03. GPT 모델

## 3.1. 모델 로드

In [ ]:
from transformers import TFGPT2LMHeadModel, PreTrainedTokenizerFast, TFAutoModelWithLMHead

In [ ]:
model = TFGPT2LMHeadModel.from_pretrained("drive/MyDrive/kogpt2", from_pt=True)
tokenizer = PreTrainedTokenizerFast.from_pretrained("drive/MyDrive/kogpt2")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight', 'transformer.h.2.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

## 3.2. koGPT2 이용 언어 생성 모형 테스트

In [ ]:
input_ids = tokenizer.encode('딥러닝', return_tensors="tf")
input_ids

<tf.Tensor: shape=(1, 4), dtype=int32, numpy=array([[ 739, 7299, 7397, 7180]], dtype=int32)>

In [ ]:
result= model.generate(input_ids,
                max_length=20,
                early_stopping=True, 
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id)
result

<tf.Tensor: shape=(1, 20), dtype=int32, numpy=
array([[  739,  7299,  7397,  7180,  8137,  9430, 12384,  8704, 30838,
        16936, 12384,  8704, 30838, 16936, 12384,  8704, 30838, 16936,
        12384,  8704]], dtype=int32)>

In [ ]:
tokenizer.decode(result[0])

'딥러닝을 통해 학습한 데이터를 기반으로 학습한 데이터를 기반으로 학습한 데이터를 기반으로 학습한'

-> 위의 결과 처름 언어 생성 모형의 한계는 뒤로 갈수록 같은말 반복을 진행 함

In [ ]:
input_ids = tokenizer.encode('사이버대학교', return_tensors="tf")

In [ ]:
result= model.generate(input_ids,
                max_length=20,
                early_stopping=True, 
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id)

tokenizer.decode(result[0])

'사이버대학교 등 전국 주요 대학들이 신입생 모집에 나서고 있다.\n이들 대학들은 신입생'

## 3.3. 챗봇 데이터의 전처리

In [ ]:
for row in df.itertuples():
  break

In [ ]:
row

Pandas(Index=0, Q='12시 땡!', A='하루가 또 가네요.', label=0)

In [ ]:
tokenizer.encode(row.Q)

[9349, 7888, 739, 7318, 376]

In [ ]:
AA = tokenizer.encode(row.Q, row.A)

In [ ]:
tokenizer.decode(AA)

'12시 땡! 하루가 또 가네요.'

In [ ]:
BB = tokenizer.encode(f'{row.Q} {row.A}')

In [ ]:
tokenizer.decode(BB)

'12시 땡! 하루가 또 가네요.'

In [ ]:
f'Q:{row.Q} A:{row.A}'

'Q:12시 땡! A:하루가 또 가네요.'

In [ ]:
data = []

for row in df.itertuples():
  tokens = tokenizer.encode(f'<s>Q: {row.Q} A: {row.A}</s>')
  data.append(tokens)

In [ ]:
len(data)

11823

### 3.4. TensorFlow가 처리 할 수 있도록 tf의 Dataset 함수로 작업

In [ ]:
import tensorflow as tf

In [ ]:
def data_generator():
    for datum in data:
        yield datum

In [ ]:
dataset = tf.data.Dataset.from_generator(data_generator, output_types=tf.int32)

In [ ]:
dataset

<FlatMapDataset element_spec=TensorSpec(shape=<unknown>, dtype=tf.int32, name=None)>

In [ ]:
len(data[10])  #11번때 대화 길이

20

In [ ]:
dataset = dataset.padded_batch(32, padded_shapes=(None, ), padding_values=tokenizer.pad_token_id)

In [ ]:
batch = next(iter(dataset))

In [ ]:
batch

<tf.Tensor: shape=(32, 32), dtype=int32, numpy=
array([[    0, 16026,   401, ...,     0,     0,     0],
       [    0, 16026,   401, ...,     0,     0,     0],
       [    0, 16026,   401, ...,     0,     0,     0],
       ...,
       [    0, 16026,   401, ...,     0,     0,     0],
       [    0, 16026,   401, ...,     0,     0,     0],
       [    0, 16026,   401, ...,     0,     0,     0]], dtype=int32)>

-> tf.Tensor를 보면 List에서 변형 되었으며, shape를 보면 1개 배치는 32건이고 최대 길인 32에 맞추어 져 있음, 그리고 0토큰을 이용해 패딩됨

##3.5. GPT2의 미세 조정

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=3e-5)

In [ ]:
result = model(batch, labels=batch)

In [ ]:
result['loss']

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([7.907617], dtype=float32)>

In [ ]:
tf.reduce_mean(result['loss'])

<tf.Tensor: shape=(), dtype=float32, numpy=7.907617>

In [ ]:
with tf.GradientTape() as tape:
    result = model(batch, labels=batch)
    loss = result['loss']

In [ ]:
grads = tape.gradient(loss, model.trainable_variables)

In [ ]:
opt.apply_gradients(zip(grads, model.trainable_variables))

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

In [ ]:
result = model(batch, labels=batch)
tf.reduce_mean(result['loss'])

<tf.Tensor: shape=(), dtype=float32, numpy=5.8463764>

위에서는 7.9에서 지금은 5.8로 줄어든걸 알 수 있음(한번 수행 했으니까)

In [ ]:
total = df.shape[0] // 32 + 1

In [ ]:
total

370

## 3.6. 학습  
위에서 학습과 최적화 과정을 반복 해주면 된다

In [ ]:
import tqdm.notebook

In [ ]:
for batch in tqdm.notebook.tqdm(dataset, total=total):
    with tf.GradientTape() as tape:
        result = model(batch, labels=batch)
        loss = result['loss']
        mean_loss = tf.reduce_mean(loss)

    grads = tape.gradient(mean_loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables))

  0%|          | 0/370 [00:00<?, ?it/s]

위에서 처름 for 문은 1에포크이다. 여러번은 한번더 for를 해야 하며,  
하지만 사전 학습된 모델의 미세 조정 할때는 많은 에포크를 하지 않는다.  
사전 모델이 망각 될 수도 있음.

In [ ]:
model.save_pretrained("/gdrive/My Drive/kogpt2/chat_model")

##3.7. 테스트
사전학습된 GPT2와 챗봇 데이터의 미세 조정을 마친 데이터로 챗봇 해보기

In [ ]:
text = '날씨가 어때?'
sent = f'<s>Q: {text} A:'

In [ ]:
sent

'<s>Q: 날씨가 어때? A:'

In [ ]:
#토큰화 진행
input_ids = tokenizer.encode(sent, return_tensors='tf')

In [ ]:
output = model.generate(input_ids, 
                        max_length=20,
                        early_stopping=True, 
                        eos_token_id=tokenizer.eos_token_id, 
                        pad_token_id=tokenizer.pad_token_id)

In [ ]:
tokenizer.decode(output[0])

'<s> Q: 날씨가 어때? A: 날씨가 좋으면 좋겠어요.</s>'

In [ ]:
answer = tokenizer.decode(output[0])
answer[len(sent):-len('</s>')]

': 날씨가 좋으면 좋겠어요.'

In [ ]:
output = model.generate(input_ids, max_length=20, 
                        do_sample=True,                            # 대답의 다양성
                        early_stopping=True, 
                        eos_token_id=tokenizer.eos_token_id, 
                        pad_token_id=tokenizer.pad_token_id)
tokenizer.decode(output[0])

'<s> Q: 날씨가 어때? A: 한 주 내내 날씨 춥다는 건 생각처럼 힘들다는'

## 저장된 모델 로드 및 실행

In [ ]:
chat_model = TFAutoModelWithLMHead.from_pretrained('/gdrive/My Drive/kogpt2/chat_model')

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_tf_auto.py:605: FutureWarning: The class `TFAutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `TFAutoModelForCausalLM` for causal language models, `TFAutoModelForMaskedLM` for masked language models and `TFAutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /gdrive/My Drive/kogpt2/chat_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
text = '뭘 드릴까요?'
sent = f'<s>Q: {text} A:'

input_ids = tokenizer.encode(sent, return_tensors='tf')

output = chat_model.generate(input_ids, max_length=50, 
                        early_stopping=True,
                        do_sample=True, 
                        eos_token_id=tokenizer.eos_token_id, 
                        pad_token_id=tokenizer.pad_token_id)
tokenizer.decode(output[0])

'<s> Q: 뭘 드릴까요? A: 지금 하아직 결정을 내릴 시간이 아니에요.</s>'